In [1]:
import csv
from typing import Set, Dict
from chgk_rating_client import Rating

In [2]:
questions_mapping_str = """
2 4 8 9 12 13 14 16 18 19 21 23 24 25 30
10 11 20 70 35 36 48 49 50 53 54 55 58 59 60
32 37 38 39 41 34 61 64 65 66 67 69 72 75
"""

questions = set(int(q) for q in questions_mapping_str.split())

In [3]:
def extract_synch_questions_results(team_result: Dict, questions: Set):
    team_result_str = team_result["mask"]
    team_relevant_results = [int(q_res) 
                             for q_num, q_res in enumerate(team_result_str, 1) 
                             if q_num in questions]
    return team_result["current_name"], sum(team_relevant_results), "кест"

def get_merged_results(base_id: int, synch_id: int, questions: Set, client: Rating):
    base_results_raw = client.tournament_results(base_id)
    synch_results_raw = client.tournament_results(synch_id)
    base_results = [extract_synch_questions_results(tr, questions) 
                    for tr in base_results_raw]
    synch_results = [(tr["current_name"], int(tr["questions_total"]), "синхрон")
                    for tr in synch_results_raw]
    return base_results + synch_results

In [4]:
base_id = 5773
synch_id = 5774
rating_client = Rating(redis_host="localhost")
merged = get_merged_results(5773, 5774, questions, rating_client)
merged_and_sorted = sorted(merged, key=lambda x: -x[1])

In [5]:
with open("kest_2019.csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(("Команда", "Сумма", "Турнир"))
    writer.writerows(merged_and_sorted)